In [ ]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import requests
from datetime import datetime, timedelta
import re
from typing import Dict, List, Tuple, Optional
from dateutil import parser as date_parser
import pytz

# Import the enhanced agentic scheduler
from agentic_scheduler import AgenticScheduler

print("🤖 Enhanced Agentic AI Scheduling Assistant - AMD Hackathon 2025")
print("✨ Now with MCP, OR-Tools optimization, and advanced AI agents!")

In [ ]:
app = Flask(__name__)
received_data = []

# Initialize Enhanced Agentic Scheduler
print("🚀 Initializing Agentic AI Scheduling System...")

# Configuration for AMD Hackathon
VLLM_SERVER_URL = "http://localhost:3000/v1"  # Adjust as needed
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"  # Adjust as needed

# Create scheduler instance with agentic capabilities enabled
scheduler = AgenticScheduler(
    vllm_url=VLLM_SERVER_URL,
    model_name=MODEL_PATH,
    agentic_mode=True  # Enable advanced AI agents and OR optimization
)

print("✅ Agentic Scheduler initialized successfully!")
print(f"🔗 Connected to vLLM server: {VLLM_SERVER_URL}")
print(f"🧠 Using model: {MODEL_PATH}")

# Test scheduler readiness
if scheduler.agentic_mode:
    print("🤖 Agentic AI Mode: ENABLED")
    print("   • Advanced information extraction agents")
    print("   • OR-Tools constraint programming optimization") 
    print("   • MCP protocol integration")
    print("   • Sub-10 second latency optimization")
else:
    print("📝 Traditional Mode: Using fallback algorithms")

In [3]:
def your_meeting_assistant(data): 
    """
    Main function called by Flask server
    This implements the Agentic AI Scheduling Assistant
    """
    return scheduler.process_meeting_request(data)

In [4]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [7]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
# Test the Agentic AI implementation
print("🧪 Testing Enhanced Agentic AI Scheduling Assistant...")
print("🎯 AMD Hackathon 2025 - Advanced Autonomous Scheduling")

# Load sample input
with open('JSON_Samples/Input_Request.json', 'r') as f:
    test_input = json.load(f)

print("\n📧 Sample Input:")
print(json.dumps(test_input, indent=2))

# Test the assistant with timing
print("\n🤖 Processing with Agentic AI Assistant...")
print("   🧠 Using advanced information extraction agents")
print("   🔍 Applying OR-Tools constraint programming optimization") 
print("   📅 Integrating with Google Calendar API")
print("   ⚡ Target: < 10 seconds end-to-end")

import time
start_time = time.time()

# Process with enhanced agentic scheduler
result = your_meeting_assistant(test_input)

end_time = time.time()
processing_time = end_time - start_time

print(f"\n⏱️ Total Processing Time: {processing_time:.3f} seconds")
if processing_time < 10:
    print("✅ LATENCY REQUIREMENT MET! (< 10 seconds)")
else:
    print("⚠️ Latency target exceeded")

print("\n📅 Generated Output:")
print(json.dumps(result, indent=2))

# Validate output format
required_fields = ['Request_id', 'EventStart', 'EventEnd', 'Duration_mins', 'Attendees']
missing = [f for f in required_fields if f not in result]

if missing:
    print(f"\n❌ Missing fields: {missing}")
else:
    print("\n✅ All required fields present!")
    print("🎯 Format validation passed!")

# Performance summary
print(f"\n🏆 AGENTIC AI PERFORMANCE SUMMARY:")
print(f"   ⚡ Response Time: {processing_time:.3f}s")
print(f"   🤖 Agent Mode: {'ENABLED' if scheduler.agentic_mode else 'DISABLED'}")
print(f"   🔧 OR-Tools: {'ACTIVE' if scheduler.agentic_ready else 'FALLBACK'}")
print(f"   📊 Meeting Optimization: COMPLETE")
print(f"   🎉 Ready for Hackathon Submission!")

# Optional: Show agent extraction details
if hasattr(result, 'agent_extracted_info'):
    print(f"\n🧠 Agent Extracted Info:")
    print(json.dumps(result.agent_extracted_info, indent=2))

🧪 Testing Agentic AI Scheduling Assistant...

📧 Sample Input:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

🤖 Processing with AI Assistant...

⏱️ Processing Time: 0.253 seconds

📅 Generated Output:
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-24T09:00:00+05:30",
          "EndTime": "2025-07-24T10:00:00+05:30",
          "NumAttendees": 1,
          "